In [ ]:
# Step 1. cafe_menu_data.txt 로드 및 분할
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
import os

# 파일 위치
data_path = "./data/cafe_menu_data.txt"

# Document 로드
loader = TextLoader(data_path, encoding="utf-8")
docs = loader.load()

# 메뉴 단위 분할
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n"],  # 메뉴 하나가 \n\n 기준으로 나눠짐
    chunk_size=500,
    chunk_overlap=0
)
split_docs = splitter.split_documents(docs)

# 임베딩 및 벡터화
embedding = OllamaEmbeddings(model="bge-m3")  # ollama pull bge-m3 필수
vectorstore = FAISS.from_documents(split_docs, embedding)

# 저장
os.makedirs("./db", exist_ok=True)
vectorstore.save_local("./db/cafe_db")

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

In [ ]:
# Step 2. 도구 정의
from langchain.tools import tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보 검색"""
    tavily = TavilySearchResults(k=3)
    return str(tavily.run(query))

@tool
def wiki_summary(topic: str) -> str:
    """위키피디아에서 요약 정보 검색"""
    wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
    return wiki.run(topic)

@tool
def db_search_cafe_func(query: str) -> list:
    """로컬 벡터 DB에서 메뉴 관련 정보 검색"""
    db = FAISS.load_local(
        "./db/cafe_db",
        OllamaEmbeddings(model="bge-m3"),
        allow_dangerous_deserialization=True
    )
    return db.similarity_search(query, k=2)

In [ ]:
# Step 3. LLM과 Tool 연결
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

agent = initialize_agent(
    tools,
    llm,
    agent_type="openai-tools",
    verbose=True
)

In [ ]:
# Step 4. 질문 테스트
query = "아메리카노의 가격과 특징은 무엇인가요?"
response = agent.run(query)
print(response)